In [ ]:
%matplotlib inline

from sklearn.datasets import make_blobs
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KDTree
import math

### First, we generate some sample data.

In [ ]:
n_samples = 1500
random_state = 170
X, _ = make_blobs(n_samples=n_samples,
                  centers=5,
                  random_state=random_state)

mins = np.amin(X, axis=0)
maxs = np.amax(X, axis=0) 


### Then we set the number of clusters k.

In [ ]:
k = 2

### Define Helper function for plotting

In [ ]:
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, k)]
old_clusters = []

def plot_kmeans(cluster_coords, ind):
    plt.figure(figsize=(10, 7))
    plt.title("k = {}, iteration = {}".format(k, iteration))
    
    # Data points
    plt.scatter(X[:, 0],
                X[:, 1],
                c=list(map(lambda x: colors[x[0]], ind)),
                s=2)
    
    # Current cluster center
    plt.scatter(cluster_coords[:, 0],
                cluster_coords[:, 1],
                marker="o",
                edgecolors='k',
                facecolors=colors,
                s=150)

    # Old cluster centers
    for old_cluster in old_clusters:
        plt.scatter(old_cluster[:, 0],
                    old_cluster[:, 1],
                    marker="o",
                    edgecolors='k',
                    facecolors=colors,
                    alpha=0.3,
                    s=100)
    # Lines
    for i in range(len(cluster_coords)):
        xs = [old_cluster[i, 0] for old_cluster in old_clusters] + [cluster_coords[i, 0]]
        ys = [old_cluster[i, 1] for old_cluster in old_clusters] + [cluster_coords[i, 1]]
        plt.plot(xs, ys, color=colors[i], alpha=0.3)

    plt.xlim(mins[0], maxs[0])
    plt.ylim(mins[1], maxs[1])
    plt.show()


### Initialize random cluster centers

In [ ]:
cluster_coords = np.random.uniform(low=mins,
                                   high=maxs,
                                   size=(k, 2))

tree = KDTree(cluster_coords)
dist, ind = tree.query(X, k=1)

### Assign data points to nearest cluster center and update cluster centers

In [ ]:
max_iterations = 30
iteration = 0
has_converged = False

plot_kmeans(cluster_coords, ind)

while not has_converged and iteration < max_iterations:
    iteration += 1
    old_clusters.append(np.copy(cluster_coords))

    # Assign 
    tree = KDTree(cluster_coords)
    dist, ind = tree.query(X, k=1)

    # Update cluster centers
    idx = ind.reshape(X.shape[0])
    for i in range(k):
        cluster_coords[i] = X[idx==i,:].mean(axis=0)

    plot_kmeans(cluster_coords, ind)

    norm = np.linalg.norm(cluster_coords - old_clusters[-1])
    print(norm)
    has_converged = math.isclose(norm, 0.0)